In [7]:
from data_handling.api import direct_download

from scipy.optimize import minimize
from scipy import stats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pystan

from functools import partial

%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 10)

## Data

In [2]:
tickers = ['IVV', 'EFA', 'EEM', 'IEF', 'USIG', 'EMB']
data = await direct_download(tickers)

In [3]:
returns = np.log(data['adj_close'].unstack()).diff().dropna()
# Stochastic volatility model assumes zero mean returns process
returns -= returns.mean()

In [5]:
returns.columns

Index(['EEM', 'EFA', 'EMB', 'IEF', 'IVV', 'USIG'], dtype='object', name='ticker')

## CAPM Equilibrium Returns

In [21]:
# Must transform log returns back to simple returns for calculation. Transforms equilbirum returns to log returns
# 1-month T-Bill constant maturity rate
# https://fred.stlouisfed.org/series/DGS1MO
rf = .0205
rf = (1 + rf) ** (1/252) - 1
# CAPM equilibrium returns.
# Must be in same order as tickers index
market_cap = [26.31, 57.94, 15.37, 17.24, 179.43, 3.48]
market_cap_weights = market_cap / np.sum(market_cap)
portfolio_returns = (np.exp(returns) - 1) @ market_cap_weights
# Fabozzi 'Robust Portfolio Optimization and Management' pg 234
risk_premium = (np.mean(portfolio_returns) - rf) / np.var(portfolio_returns)
equil_return = ((risk_premium * np.cov((np.exp(returns) - 1).dropna().T)) @ market_cap_weights) + rf
# Transform into log return
equil_return = np.log(equil_return + 1)
pd.DataFrame(equil_return, index=returns.columns)

,0
ticker,
EEM,0.000085
EFA,0.000084
EMB,0.000081
IEF,0.000080
IVV,0.000084
USIG,0.000081


## Return Sampling

In [9]:
# 6 Tickers, 4 chains, 1000 iterations, 250 warmup on 4790k: 1.8 hours
# 6 Tickers, 4 chains, 2750, 250 warmup, adapt_delt: 0.90 on 4790k: 3.91 hours
# 6 Tickers, 4 chains, 1500 iterations, 250 warmup, adapt_delta: 0.90 on 4790k: 2.08 hours
# adapt_delta: 0.90 results in significantly better effective samples
# 0.2475 R-squared calculated from squared returns as volatility
# 'MULTIVARIATE STOCHASTIC VOLATILITY MODELS:BAYESIAN ESTIMATION AND MODEL COMPARISON' Jun Yu, Renate Meyer 2006
# Constant Correlation Model
model_spec = '''
data {
    int len;                  // Length of time series
    int num;                  // Number of tickers
    vector[num] returns[len]; // Matrix of size (len, num)
    corr_matrix[num] corr;    // Correlation matrix
}
transformed data {
    cholesky_factor_corr[num] L;
    L = cholesky_decompose(corr);
}
parameters {
    vector[num] mu;
    vector<lower=-1, upper=1>[num] phi;
    vector<lower=0>[num] sigma;
    vector[num] h_std[len];
}
transformed parameters {
    vector[num] h[len];        // Distributed normal(mu + phi * (h[t-1] - mu), sigma)
    for (t in 1:len) {
        if (t == 1) {
            h[t] = h_std[t] .* sigma;
            h[t] += mu;
        }
        else {
            h[t] = h_std[t] .* sigma;
            h[t] += mu + phi .* (h[t-1] - mu);
        }
    }
}
model {
    mu ~ normal(0, 25);
    phi ~ uniform(-1, 1);
    sigma ~ inv_gamma(2.5, .1);
    
    for (t in 1:len) {
        h_std[t] ~ std_normal();
        // Cholesky of covariance matrix == diag matrix of standard deviations * Cholesky of correlation matrix
        returns[t] ~ multi_normal_cholesky(rep_vector(0, num), diag_pre_multiply(exp(h[t] / 2), L));
    }
}
'''

In [10]:
model = pystan.StanModel(model_code=model_spec)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_aeb0d0c059ad7094d680b2ca5566f00a NOW.


In [15]:
def init_func():
    length = len(returns.columns)
    init_dict = {'mu': np.full(length, -10), 'phi': np.full(length, 0.90), 'sigma': np.full(length, 0.10)}
    return init_dict

In [16]:
# Multivariate
params = {'len': len(returns), 'num': len(returns.columns), 'returns': returns, 'corr': np.corrcoef(returns.T)}

control = {'adapt_delta': 0.90}
mcmc_sample = model.sampling(data=params, chains=4, warmup=250, iter=1500, control=control, init=init_func)

To run all diagnostics call pystan.check_hmc_diagnostics(fit)


In [19]:
sample_data = mcmc_sample.extract(pars=['mu', 'phi', 'sigma', 'h'], permuted=False)

In [20]:
# Saves MCMC samples
import pickle
with open('sample_data', 'wb') as file:
    pickle.dump(sample_data, file, protocol=pickle.HIGHEST_PROTOCOL)

In [22]:
# Aggreate data across each chain into single dimension
mu = sample_data['mu'].reshape(-1, len(returns.columns))
phi = sample_data['phi'].reshape(-1, len(returns.columns))
sigma = sample_data['sigma'].reshape(-1, len(returns.columns))
# Volatility has additional time dimension
h = sample_data['h'].reshape(-1, len(returns), len(returns.columns))

# Generate 100,000 random indexes
idx = np.random.randint(0, 5000, size=100000)

# Get random samples for each parameter
mu = mu[idx]
phi = phi[idx]
sigma = sigma[idx]
# Select only last volatility entry, removing time dimension
h = h[idx, -1]

In [23]:
# Forward simulation
h_sim = np.zeros((21, 100000, 6))
returns_sim = np.zeros((21, 100000, 6))
cholesky_corr = np.linalg.cholesky(np.corrcoef(returns.T))
for t in range(21):
    # First iteration use last volatility from MCMC samples
    if t == 0:  
        h_sim[t] = mu + phi * (h - mu) + np.random.normal(loc=0, scale=sigma, size=(100000, len(returns.columns)))
        # First Creates array of size (100000, len, len): sequence of diagonal matrices created from volatility
        # diagonal matrix of standard deviations * cholesky of correlation matrix = cholesky of covariance matrix
        cholesky_mats = (np.eye(len(returns.columns)) * np.exp(h_sim[t] / 2)[:, np.newaxis, :]) @ cholesky_corr
        # Multiples each of those matrices by a vector of standard normal samples to create correlated samples
        # Cholesky of covariance * vector of standard normals = correlated multivariate normal
        # I really don't understand how einsum works, but this DOES WORK PROPERLY
        returns_sim[t] = np.einsum('ijk,ik->ij', cholesky_mats, np.random.normal(loc=0, scale=1, size=(100000, len(returns.columns))))
    # Rest of iterations use last simulated volatility sample
    else:
        h_sim[t] = mu + phi * (h_sim[t - 1] - mu) + np.random.normal(loc=0, scale=sigma, size=(100000, len(returns.columns)))
        cholesky_mats = (np.eye(len(returns.columns)) * np.exp(h_sim[t] / 2)[:, np.newaxis, :]) @ cholesky_corr
        returns_sim[t] = np.einsum('ijk,ik->ij', cholesky_mats, np.random.normal(loc=0, scale=1, size=(100000, len(returns.columns))))

In [24]:
# Create sample for optimization.
# Add in expected daily return
sample = returns_sim + equil_return
# Simulated returns are logarithmic so sum accross time for total return over period
sample = np.sum(sample, axis=0)
# Convert to simple returns
sample = np.exp(sample) - 1

## Optimization

In [29]:
def cvar(weights, var_level, sample):
    """
    Returns negative CVaR for minimization
    weights: vector indicating portfolio weights
    var_level: (0, 100) number indicating VaR level to use
    sample: (n, m) matrix containing asset returns sample where m is number of assets and n is number of samples
    """
    port_returns = sample @ weights
    var = np.percentile(port_returns, (100 - var_level))
    return -np.mean(port_returns[port_returns < var])

In [30]:
def opt_fun(weights, sample):
    """
    Returns negative portfolio return for minimization.
    Uses L2 regularization to stabalize portfolio weights across differing samples.
    Without regularization weights display high variance from small variances in sampling.
    weights: m length vector of portfolio weights where m is number of assets
    sample: (n, m) matrix containing asset return samples where m is number of assets and n is number of samples
    """
    port_returns = sample @ weights
    # This regularization coefficient makes weights vary within about +- 1% in my testing
    return -np.mean(port_returns) + .001 * np.sqrt(np.sum(np.square(weights)))

In [31]:
# Fixing parameters with partial functions
cvar_partial = partial(cvar, var_level=99, sample=sample)
opt_fun_partial = partial(opt_fun, sample=sample)
# Maximize return subject to CVaR constraint
constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}, # Sum of weights should be 1
               {'type': 'ineq', 'fun': lambda x: .05 - cvar_partial(x)}] # CVaR should be <= to some value
bounds = [(0, 1) for x in returns.columns] # Long only
guess = np.ones(len(returns.columns)) / len(returns.columns) # Intial guess equal weight
# Increased tolerance is needed to handle small daily return values
result = minimize(opt_fun_partial, guess, constraints=constraints, bounds=bounds, method='SLSQP', tol=1e-10)

In [1]:
print(f'99% VaR: {round(np.percentile(sample @ result.x, 1) * 100, 2)}%')
print(f'CVaR: {round(-cvar_partial(result.x) * 100, 2)}%')
print(f'Montly Return: {round(np.mean(sample @ result.x) * 100, 2)}%')
print(f'Annualized Return: {round(((1 + np.mean(sample @ result.x))**12 - 1) * 100, 2)}%')
pd.DataFrame(result.x, index=returns.columns).round(3) * 100

NameError: name 'np' is not defined

# Shares Conversion

In [33]:
port_value = 5386.91

In [34]:
last_prices = data['adj_close'].unstack().iloc[-1]
# Number of shares needed to meet optimized allocation, always rounded down to nearest integer
np.floor(result.x * port_value / last_prices)

ticker
EEM     29.0
EFA      6.0
EMB      8.0
IEF     13.0
IVV      1.0
USIG    14.0
Name: 2019-08-07 00:00:00, dtype: float64